<a href="https://colab.research.google.com/github/Samin-Sadaf7/QuestionAnswerGenerator/blob/main/Testing_images_for_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Document parsing and image summarization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/AI_integration_kernel'

/content/drive/MyDrive/AI_integration_kernel


#Document Parsing

In [ ]:
!pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt-get install poppler-utils

In [ ]:
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-script-latn

In [ ]:
!pip install unstructured-pytesseract
!pip install tesseract-ocr

In [ ]:
from unstructured.partition.pdf import partition_pdf

In [ ]:
res_pdf_elements =partition_pdf(
    filename ="/content/drive/MyDrive/AI_integration_kernel/vishram Singh upper limb and thorax v1-1.pdf",
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_to_payload=False,
    extract_table_block_output_dir="extracted_data"
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

In [ ]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]

for element in res_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
    Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
    Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
    Title.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
    Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
    ListItem.append(str(element))
  elif 'unstructured.documents.elements.NarrativeText' in str(type(element)):
    NarrativeText.append(str(element))

In [ ]:
img =[]
for element in res_pdf_elements:
  if "unstructured.documents.elements.Image" in str(type(element)):
    img.append(str(element))

In [ ]:
table=[]
for element in res_pdf_elements:
  if "unstructured.documents.elements.Table" in str(type(element)):
    table.append(str(element))

#Start with open ai

In [ ]:
!pip install langchain_core

In [ ]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.8 MB/s eta 0:00:00


In [ ]:
import shutil

# Source path
src_path = '/content/figures'

# Destination path
dest_path = '/content/drive/MyDrive/AI_integration_kernel'

# Move folder
shutil.move(src_path, dest_path)

print(f"Folder moved from {src_path} to {dest_path}")


Folder moved from /content/figures to /content/drive/MyDrive/AI_integration_kernel


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
!pip install google-ge

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OpenAIKey')
os.environ['GEMINI_API_KEY'] = userdata.get('GeminiKey')
os.environ['GOOGLE_API_KEY'] = userdata.get('GeminiKey')

In [ ]:
prompt_text="""You are an assistant tasked with summarizing tables for retrieval. Theses summaries will be embedded and used to retreive the raw tables elements. Give a conscise summary of the table that is well optimized for retrieval. Table {element}"""

In [ ]:
prompt= ChatPromptTemplate.from_template(prompt_text)

In [ ]:
model = ChatOpenAI(temperature=0.9, model="gpt-4")

In [ ]:
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
table_summaries=[]

In [ ]:
table[0]

'Preface to the Second Edition Preface to the First Edition Acknowledgments Introduction to the Upper Limb Shoulder Joint Complex (Joints of Shoulder Girdle) Joints and Movements of the Hand Introduction to Th orax and Th oracic Cage vii ix xi 1 10 34 48 58 72 83 92 105 126 137 161 172 185 196 211 227 234 249'

In [ ]:
import base64
import os
from langchain_core.messages import HumanMessage

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


In [ ]:
def image_summarize(img_base64, prompt):
  chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens =1024)
  msg = chat.invoke(
      [
          HumanMessage(
              content=[
                  {"type":"text", "text":prompt},
                  {"type":"image_url",
                   "image_url": {"url":f"data:image/png;base64,{img_base64}"}
                   },
              ]
          )
      ]
  )
  return msg.content

In [ ]:
def generate_img_summaries(path):
    img_base64_list= []
    image_summaries = []
    prompt = """You are an assistant tasked with summarizing images for retrieval. Theses summaries will be embedded and used to retreive the raw image. Give a conscise summary of the image that is well optimized for retrieval."""
    base64_image = encode_image(path)
    img_base64_list.append(base64_image)
    image_summaries.append(image_summarize(base64_image, prompt))
    return img_base64_list, image_summaries

In [ ]:
!pip install langchain-community sentence-transformers pypdf2 pypdf chromadb
!pip install google-generativeai

In [ ]:
!pip install --upgrade --quiet  langchain-google-genai pillow

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
model= ChatGoogleGenerativeAI(model="gemini-pro")

In [ ]:
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
table[0]

In [ ]:
tables_summaries= summarize_chain.batch(table, {"max_concurrency":5})

KeyboardInterrupt: 

In [ ]:
img[0]

In [ ]:
img_base64_list , generate_img_summaries = g